#Cleaning Data
## Por: José Guarnizo
Nombre de la variable	Descripción
* Item_Identifier	Número de identificación único del producto
* Item_Weight	Peso del producto
* Item_Fat_Content	Si el producto es bajo en grasas o regular
* Item_Visibility	El porcentaje de área total de visualización de todos los productos en la tienda asignados a este producto particular
* Item_Type	La categoría a la que pertenece el producto
* Item_MRP	Precio Máximo Minorista (precio de lista) del producto
* Outlet_Identifier	Número de identificación único de la tienda
* Outlet_Establishment_Year	El año en el que se estableció la tienda
* Outlet_Size	El tamaño de la tienda en cuanto a la superficie total que cubre
* Outlet_Location_Type	El tipo de área donde se ubica la tienda
* Outlet_Type	Si la tienda es un almacén o algún tipo de supermercado
* Item_Outlet_Sales	Las ventas del producto en la tienda particular. Esta es la * variable objetivo que se debe predecir.

##Lectura de archivo sales_predictions.csv
* Conexión con Google.Colab
* Se guarda archivo en variable df

In [269]:
import pandas as pd

In [274]:
filename='https://raw.githubusercontent.com/JGuarnizo/Sales-Predictions/51d233017c983387a05c8d0f96e793e1913002f6/sales_predictions.csv'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [275]:
df=pd.read_csv(filename)

## Inspección del data frame
* Existen 8523 observaciones. 
* Hay 5 variables númericas y 6 categoricas
* Se encuentran 2 columnas con datos faltantes. 
* Se obtiene porcentaje de datos faltantes
1. Item_Weight (Float) 17.2%
2. Outlet_Size (Object) 28.2%


In [73]:
df.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [10]:
#Información del data frame
#Existen 2 columnas con datos faltantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [11]:
#Se obtiene un porcentaje de datos faltantes
round((df.isna().sum()/len(df))*100,1)

Item_Identifier               0.0
Item_Weight                  17.2
Item_Fat_Content              0.0
Item_Visibility               0.0
Item_Type                     0.0
Item_MRP                      0.0
Outlet_Identifier             0.0
Outlet_Establishment_Year     0.0
Outlet_Size                  28.3
Outlet_Location_Type          0.0
Outlet_Type                   0.0
Item_Outlet_Sales             0.0
dtype: float64

## Rellenar valores Outlet_Size
* La variable Outlet_Size esta ligada a la variable Outlet_Identifier. Por o tanto, se agrupa por Outlet_Identifier y se observa que 3 tiendas no tienen definido su Outlet_Size. La tiendas son: OUT010, OUT017 Y LA OUT045
* Se intenta encontrar alguna relación entre Outlet_Size y Outlet_type para rellenar valores faltantes, pero no hay ninguna relación.
* Si se borran todas las obsrvaciones donde Outlet_Size==NaN entonces se borraría la información de 3 tiendas, por lo tanto no aplica.
* CONCLUSIÓN: Outlet_Size es una variable categórica que no aporta mayor información. Como no se tiene parámetros de como fue creada, se desconoce a que tamaño se refiere Small, Medium o High. Se puede eliminar pero en este caso la ignoraremos en nuestro data frame.

In [229]:
#Outlet_Size es el tamaño de cada tienda. Existe un número limitado de tiendas y cada tienda debe tener un tamaño fijo. 
df.groupby(['Outlet_Identifier'])['Outlet_Size'].min()

Outlet_Identifier
OUT010       NaN
OUT013      High
OUT017       NaN
OUT018    Medium
OUT019     Small
OUT027    Medium
OUT035     Small
OUT045       NaN
OUT046     Small
OUT049    Medium
Name: Outlet_Size, dtype: object

In [236]:
df.groupby(['Outlet_Identifier'])['Outlet_Size','Outlet_Type'].min()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Outlet_Size,Outlet_Type
Outlet_Identifier,,
OUT010,NaN,Grocery Store
OUT013,High,Supermarket Type1
OUT017,NaN,Supermarket Type1
OUT018,Medium,Supermarket Type2
OUT019,Small,Grocery Store
OUT027,Medium,Supermarket Type3
OUT035,Small,Supermarket Type1
OUT045,NaN,Supermarket Type1
OUT046,Small,Supermarket Type1


In [228]:
df.groupby(['Outlet_Identifier'])['Item_Outlet_Sales'].sum()

Outlet_Identifier
OUT010    1.883402e+05
OUT013    2.142664e+06
OUT017    2.167465e+06
OUT018    1.851823e+06
OUT019    1.796941e+05
OUT027    3.453926e+06
OUT035    2.268123e+06
OUT045    2.036725e+06
OUT046    2.118395e+06
OUT049    2.183970e+06
Name: Item_Outlet_Sales, dtype: float64

##Rellenar valores Item_Weight
* Enfoque 1:
Se obtiene la lista única de Tiendas en la base de datos (Outlet_Identifier). Después se realizará un filtro para eliminar todas las observaciones donde Item_Weight==NaN, y se revisará si la lista única de Tiendas permanece igual.
* Conclusión:
Si se eliminan todos los valores donde Item_Weight==NaN se elimina la información completa de 2 tiendas. Por lo tanto, este enfoque no se aplica
* Enfoque 2:
Se conoce que se tiene el 17% de datos perdidos en la columna Item_Weight, sin embargo, el Item_Weight esta asociado a un Item_Identifier, y estos (Item_Identifier) se repiten para 1 o más tiendas.
Primero se agrupará por Item_Identifier y se analizará cuantos elementos no tienen Item_Weight en toda la base. Se analizará cuál es el impacto de venta en la Item_Outlet_Sales en cada tienda que se venda y si el porcentaje es despreciable menor a 0.5% se eliminarán dichas observaciones.
Luego, para todos los valores restantes se puede ordenar la base de datos por Item_Identifier y copiar los valores con el comando fillna(method='bfill').

* Conclusión:
Se eliminan 4 filas (observaciones) de la base de datos y el resto se rellena con el comando fillna(method='bfill') previamente ordenando la base de datos por Item_Identifier. 
* La nueva base de datos funcional queda como df_final

###Enfoque 1:

In [129]:
#Tiendas existentes en la base de datos
#Existen 10 tiendas en total
df['Outlet_Identifier'].value_counts()

OUT027    935
OUT013    932
OUT049    930
OUT035    930
OUT046    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64

In [130]:
#Si se decide eliminar todos las observaciones donde Item_Weight==NaN, entonces se eliminaría la información completa de 2 tiendas
#OUT027 & OUT019 ya no aparecen en la base filtrada.
#Por lo tanto, no se puede borrar todas las observaciones
filtro = df['Item_Weight'].isna() 
df.loc[~filtro,:]['Outlet_Identifier'].value_counts()

OUT013    932
OUT049    930
OUT035    930
OUT046    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
Name: Outlet_Identifier, dtype: int64

###Enfoque 2:

In [238]:
#Se determina cuántos Item_Identifier realmente no tienen Item_Weight
empty_arr=df.groupby(['Item_Identifier'])['Item_Weight'].mean().isna()
empty_weight_item=[]

for i in range(len(empty_arr)):
  if empty_arr[i]==True:
    empty_weight_item.append(empty_arr.index[i])

empty_weight_item

['FDE52', 'FDK57', 'FDN52', 'FDQ60']

In [268]:
#Se crea un filtro para saber cuantas registros con Item_Identifier son iguales a los valores de ['FDE52', 'FDK57', 'FDN52', 'FDQ60']
#Se observa que los 4 items sólo son vendidos en 2 tiendas en la OUT027 & OUT019
filtro=(df['Item_Identifier']==empty_weight_item[0]
        )|(df['Item_Identifier']==empty_weight_item[1]
        )|(df['Item_Identifier']==empty_weight_item[2]
        )|(df['Item_Identifier']==empty_weight_item[3])

df_empty_weight=df.loc[filtro,:]
df_empty_weight.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


In [240]:
#Ventas totales por tienda
total_sales=df.groupby(['Outlet_Identifier'])['Item_Outlet_Sales'].sum()

#Porcentaje de ventas de los items de empty_weight_item en sus respectivas tiendas
for i in df_empty_weight.index:
  print('\nEl',df_empty_weight.loc[i,'Item_Identifier'],'en la tienda',df_empty_weight.loc[i,'Outlet_Identifier'])
  print((df_empty_weight.loc[i,'Item_Outlet_Sales']/total_sales[df_empty_weight.loc[i,'Outlet_Identifier']])*100,'% de las ventas totales')



El FDN52 en la tienda OUT027
0.04545425630533231 % de las ventas totales

El FDK57 en la tienda OUT027
0.12838225063338135 % de las ventas totales

El FDE52 en la tienda OUT027
0.09998779790320557 % de las ventas totales

El FDQ60 en la tienda OUT019
0.06706386258206985 % de las ventas totales


In [241]:
#Se elimina las observaciones anteriores
df_final=df.loc[~filtro,:]
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8519 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8519 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8519 non-null   object 
 3   Item_Visibility            8519 non-null   float64
 4   Item_Type                  8519 non-null   object 
 5   Item_MRP                   8519 non-null   float64
 6   Outlet_Identifier          8519 non-null   object 
 7   Outlet_Establishment_Year  8519 non-null   int64  
 8   Outlet_Size                6109 non-null   object 
 9   Outlet_Location_Type       8519 non-null   object 
 10  Outlet_Type                8519 non-null   object 
 11  Item_Outlet_Sales          8519 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.2+ KB


In [242]:
#Se ordena df_final por Item_Identifier
df_final=df_final.sort_values(by=['Item_Identifier'])
#Se completa los valores faltantes de Item_Weight
df_final.loc[:,'Item_Weight']=df_final.loc[:,'Item_Weight'].fillna(method='bfill')
#Se ordena df_final por índice
df_final=df_final.sort_index()
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8519 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8519 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8519 non-null   object 
 3   Item_Visibility            8519 non-null   float64
 4   Item_Type                  8519 non-null   object 
 5   Item_MRP                   8519 non-null   float64
 6   Outlet_Identifier          8519 non-null   object 
 7   Outlet_Establishment_Year  8519 non-null   int64  
 8   Outlet_Size                6109 non-null   object 
 9   Outlet_Location_Type       8519 non-null   object 
 10  Outlet_Type                8519 non-null   object 
 11  Item_Outlet_Sales          8519 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.2+ KB


## Análisis de datos
* Variables númericas: 'Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales'
* Lista de tipos de elementos que se venden en las distintas tiendas
* Lista ordenada de mayor a menor de las tiendas que más venden
* Lista ordenada de mayor a menor de los tipos de productos que más se venden.

In [264]:
# Valores mínimos, máximos, desviación estándar de variables numéricas
df_final.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8519.000000,8519.000000,8519.000000,8519.000000,8519.000000
mean,12.878478,0.066112,141.010019,1997.837892,2181.188779
std,4.653350,0.051586,62.283594,8.369105,1706.511093
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.780000,0.026983,93.844900,1987.000000,834.247400
50%,12.650000,0.053925,143.047000,1999.000000,1794.331000
75%,16.850000,0.094558,185.676600,2004.000000,3100.630600
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


* La variable Outlet_Establishment_Year, es el año de apertura de la tienda y no nos aporta mayor información para las ventas.
* La variable Item_Outlet_Sales tiene valores bastante dispersos, pero, es debido a la diferencia de los productos que se vende.


In [265]:
#Obtenemos lista de tipos de elementos que se venden en las distintas tiendas
df['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [266]:
#Lista ordenada de mayor a menor de las tiendas que más venden
df.groupby(['Outlet_Identifier'])[['Item_Outlet_Sales']].sum().sort_values(by='Item_Outlet_Sales', ascending=False)

,Item_Outlet_Sales
Outlet_Identifier,
OUT027,3.453926e+06
OUT035,2.268123e+06
OUT049,2.183970e+06
OUT017,2.167465e+06
OUT013,2.142664e+06
OUT046,2.118395e+06
OUT045,2.036725e+06
OUT018,1.851823e+06
OUT010,1.883402e+05


In [267]:
#Lista ordenada de mayor a menor de los tipos de productos que más se venden.
df.groupby(['Item_Type'])[['Item_Outlet_Sales']].sum().sort_values(by='Item_Outlet_Sales', ascending=False)

,Item_Outlet_Sales
Item_Type,
Fruits and Vegetables,2.820060e+06
Snack Foods,2.732786e+06
Household,2.055494e+06
Frozen Foods,1.825735e+06
Dairy,1.522594e+06
Canned,1.444151e+06
Baking Goods,1.265525e+06
Health and Hygiene,1.045200e+06
Meat,9.175656e+05
